In [41]:
from csvObject import CsvObject, write_csv
from miscSupports import terminal_time
from pathlib import Path
import numpy as np
import pandas as pd

construction_directory = r"Z:\UKB\GeographicID\Paper Data Extraction\Construction Files"

In [12]:
# Load the data into memory
phenotypes = CsvObject(Path(construction_directory, "MilkSkinHair.csv"))
pcs = CsvObject(Path(construction_directory, "IEU_PCs_1_40.txt"), file_headers=False)

# Create a linker and gender lookup dict
linker = {app: ieu for ieu, app in  CsvObject(Path(construction_directory, "Linker.csv")).row_data}
gender_lookup = {row[0]: row[1] for row in CsvObject(Path(construction_directory, "Gender.csv")).row_data}
print(terminal_time())

15:15


In [22]:
pc_dict = {row[1]: [r for r in row[2:] if r != ""] for row in pcs.row_data}
print(terminal_time())

15:19


In [28]:
indexes = [1, 4, 7]
phenotype_rows = []
for row in phenotypes.row_data:
    try:
        phenotype_rows.append([linker[row[0]], linker[row[0]], gender_lookup[row[0]]] + [row[i] for i in indexes] + pc_dict[linker[row[0]]])
    except KeyError:
        pass
print(len(phenotype_rows))

write_csv(r"Z:\UKB\GeographicID\Paper Data Extraction\SB_Papers\SW_GWAS\GWAScompare", "NoRes",
          ["FID", "IID", "Gender", "SkinColour", "HairColour", "MilkConsumption"] + [f"PC{i}" for i in range(1, 41)],
          phenotype_rows)
print(terminal_time())

488265
15:32


In [30]:
with_residuals = pd.read_csv(r"Z:\UKB\GeographicID\Paper Data Extraction\SB_Papers\SW_GWAS\GWAScompare\Res.csv")

print(terminal_time())

15:57


## Isolate the variables we want to run a GWAS specification on

Here we isolate the variables that are covariants and our phenotypes

In [45]:
exposure_headers = list(with_residuals.columns[:8])
covariant_headers = ["fid", "iid", "gender"] + list(with_residuals.columns[8:])

In [46]:
project_directory = r"Z:\UKB\GeographicID\Paper Data Extraction\SB_Papers\SW_GWAS\GWAScompare"

disease_df = with_residuals[exposure_headers]
disease_df.columns = [h.upper() for h in exposure_headers]
disease_df = disease_df.replace({np.NaN: "NA"}, regex=True)
disease_df = disease_df.astype(str)
disease_df.to_csv(Path(project_directory, "ResCompareExposures.txt"), sep=" ", index=False)
print(f"Wrote the Exposures file at {terminal_time()}")

covariant_df = with_residuals[covariant_headers]
covariant_df.columns = [h.upper() for h in covariant_headers]
covariant_df = covariant_df.replace({np.NaN: "NA"}, regex=True)
covariant_df = covariant_df.astype(str)
covariant_df.to_csv(Path(project_directory, "ResCompareCovariant.txt"), sep=" ", index=False)
print(f"Wrote the Covariant file at {terminal_time()}")

Wrote the Covariant file at 16:15
